In [ ]:
import pandas as pd

In [ ]:
data = pd.read_parquet("train_dataset/logs_df_2024-08-05.parquet")

In [ ]:
#test = pd.read_parquet("train_dataset/logs_df_2024-08-06.parquet")

In [ ]:
#data.head()

In [ ]:
stats = pd.read_parquet("train_dataset/video_stat.parquet")

In [ ]:
data

In [ ]:
merged_data = pd.merge(data, stats, on='video_id', how='left')
merged_data

In [ ]:
merged_data['video_id'].nunique()

In [ ]:
stats['video_id'].nunique()

In [ ]:
merged_data.columns

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

merged_data['v_duration'] = merged_data['v_duration'].replace(0, 1e-6)  # Избегаем деления на ноль

# Вычисляем таргет: отношение времени просмотра к длительности видео
merged_data['target'] = merged_data['watchtime'] / merged_data['v_duration']

# Если watchtime превышает длительность видео, то таргет ограничивается значением 1
merged_data['target'] = merged_data['target'].apply(lambda x: min(x, 1))

categorical_columns = ['region', 'city', 'category_id', 'author_id']

# Применяем LabelEncoder ко всем категориальным столбцам
label_encoders = {}
for column in categorical_columns:
    label_encoders[column] = LabelEncoder()
    merged_data[column] = label_encoders[column].fit_transform(merged_data[column])

merged_data = merged_data.dropna()

# Убираем старые категориальные фичи
features = merged_data.drop(columns=['event_timestamp', 'user_id', 'video_id', 'watchtime', 
                              'v_pub_datetime', 'title', 'description', 'target', 
                              'city', 'category_id', 'author_id', 'region'])

# Разделяем данные на тренировочные и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(features, merged_data['target'], test_size=0.2, random_state=42)


In [ ]:
merged_data

In [ ]:
features

In [ ]:
import lightgbm as lgb


# Инициализация модели LightGBM Regressor
model = lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    num_leaves=31,
    learning_rate=0.05,
    n_estimators=1000,
    random_state=42
)

# Обучение модели с ранним прекращением
model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_test, y_test)],
    eval_metric='rmse',
    # early_stopping_rounds=50,
    categorical_feature=categorical_columns,
    # verbose=100
)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Вычисление метрик
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"RMSE на тестовой выборке: {rmse:.4f}")
print(f"R² на тестовой выборке: {r2:.4f}")


In [ ]:
errors = y_pred - y_test
sns.histplot(errors, bins=50, kde=True)
plt.title('Распределение ошибок предсказания')
plt.xlabel('Ошибка (Предсказание - Реальные значения)')
plt.ylabel('Частота')
plt.show()


In [ ]:

candidates_df = pd.read_parquet('candadates.parquet')

for column in categorical_columns:
    if candidates_df[column].dtype == 'object':
        candidates_df[column] = candidates_df[column].fillna('Unknown')
    else:
        candidates_df[column] = candidates_df[column].fillna(candidates_df[column].median())

for column in candidates_df.columns:
    if candidates_df[column].dtype in ['int64', 'float64']:
        candidates_df[column] = candidates_df[column].fillna(candidates_df[column].median())
    elif candidates_df[column].dtype == 'object':
        candidates_df[column] = candidates_df[column].fillna('Unknown')

X_candidates = candidates_df.drop(columns=['user_id', 'video_id'])

candidates_df['score'] = model.predict(X_candidates)
candidates_df_sorted = candidates_df.sort_values(by=['user_id', 'score'], ascending=[True, False])
candidates_df_sorted['rank'] = candidates_df_sorted.groupby('user_id')['score'].rank(method='first', ascending=False).astype(int)

print("Отсортированные кандидаты с рангами:")
print(candidates_df_sorted)

candidates_df_sorted.to_csv('ranked_candidates.csv', index=False)
